In [1]:
import pandas as pd
import numpy as np


In [3]:
df=pd.read_excel('drug_inventory.xlsx')

In [4]:
df.head()

,drug_name,units_in_stock,units_dispensed,unit_threshold,last_updated
0,Baclofen,391,18,89,2025-06-07
1,Albendazole,425,15,94,2025-06-11
2,Tramadol,190,16,25,2025-06-06
3,Fluconazole,222,78,72,2025-06-09
4,Atorvastatin,404,51,32,2025-06-05


In [11]:
#sum of units dispensed per drugs

weekly_usage = df.groupby('drug_name')['units_dispensed'].sum().reset_index()
weekly_usage.rename(columns={'units_dispensed':'Total_units_dispensed'}, inplace=True)
weekly_usage=weekly_usage.sort_values(by='Total_units_dispensed',ascending=False)

weekly_usage.head(10)
                   

,drug_name,Total_units_dispensed
47,Tramadol,890
44,Sertraline,858
29,Lisinopril,821
48,Warfarin,793
11,Ciprofloxacin,786
37,Omeprazole,786
9,Cetirizine,723
25,Ibuprofen,673
2,Amiodarone,667
5,Atorvastatin,660


In [13]:
#let's see the restocking needs for the store
df['restocking_needed']= df['units_in_stock'] < df['unit_threshold']
df['restocking_needed']=df['restocking_needed'].apply(lambda x: 'Yes' if x else 'Not Yet')

df.head(10)

,drug_name,units_in_stock,units_dispensed,unit_threshold,last_updated,restocking_needed
0,Baclofen,391,18,89,2025-06-07,Not Yet
1,Albendazole,425,15,94,2025-06-11,Not Yet
2,Tramadol,190,16,25,2025-06-06,Not Yet
3,Fluconazole,222,78,72,2025-06-09,Not Yet
4,Atorvastatin,404,51,32,2025-06-05,Not Yet
5,Ciprofloxacin,162,24,38,2025-06-06,Not Yet
6,Warfarin,465,7,62,2025-06-08,Not Yet
7,Enalapril,173,62,94,2025-06-09,Not Yet
8,Gabapentin,91,4,72,2025-06-10,Not Yet
9,Cetirizine,101,68,23,2025-06-06,Not Yet


In [17]:
#restocking alert and name of the drugs in particular
latest_stock=df.sort_values('last_updated').drop_duplicates('drug_name', keep='last')
restock_alert = latest_stock[latest_stock['units_in_stock']<latest_stock['unit_threshold']]

#print(latest_stock.head())
#print(restock_alert.head())



latest_stock.head()
restock_alert.head()

,drug_name,units_in_stock,units_dispensed,unit_threshold,last_updated,restocking_needed
324,Hydrochlorothiazide,37,9,62,2025-06-11,Yes
29,Warfarin,53,83,86,2025-06-11,Yes


In [49]:
weekly_summary=df.groupby('drug_name')[['units_dispensed', 'units_in_stock']].sum().reset_index()

weekly_summary.rename(columns={'units_dispensed':'Total_Units_dispensed', 'units_in_stock':'Total_units_in_stock'
                              }, inplace=True)

weekly_summary.head()



,drug_name,Total_Units_dispensed,Total_units_in_stock
0,Albendazole,161,834
1,Allopurinol,547,1939
2,Amiodarone,667,3782
3,Amlodipine,417,2635
4,Amoxicillin,279,2487


In [51]:
Top5_dispensed = (df.groupby('drug_name')['units_dispensed'].sum().reset_index().rename(columns={'units_dispensed':'Total_Unit_dispensed'})
                .sort_values(by='Total_Unit_dispensed', ascending=False).head(5)
                 )

print('Top 5 Most Dispensed Drugs this Week:')
Top5_dispensed.head()

Top 5 Most Dispensed Drugs this Week:


,drug_name,Total_Unit_dispensed
47,Tramadol,890
44,Sertraline,858
29,Lisinopril,821
48,Warfarin,793
11,Ciprofloxacin,786


In [52]:
with pd.ExcelWriter('Weekly_drug_report.xlsx', engine='openpyxl') as writer:
    weekly_summary.to_excel(writer, sheet_name='pharmacy_summary', index= False)
    Top5_dispensed.to_excel(writer, sheet_name='dispensed', index =False)
    restock_alert.to_excel(writer, sheet_name='Restock_alert', index = False)

print('report generated:weekly_drug_report.xlsx')

report generated:weekly_drug_report.xlsx


In [2]:
#let's email the weekly report and restocking needs to the chief-pharmacist
import smtplib 
from email.message import EmailMessage
import os

In [56]:
#Set up email details
sender  = 'kehindeolaide24@gmail.com'
receiver ='daramolarichard@gmail.com'
subject = 'weekly_drug_report'
body ='Attached is the weekly drug inventory report. restocking is required for some drugs.'
    
#Create the e-mail    
msg         = EmailMessage()
msg['To']   = receiver
msg['From'] = sender
msg['Subject'] =subject
msg.set_content(body)

#attach the excel file
with open('weekly_drug_report.xlsx', 'rb') as f:
     file_data = f.read()
     file_name = f.name
     
msg.add_attachment(file_data, 
                   maintype ='application', subtype = 'vnd.openxmlformats-officedocument.spreadsheetml.sheet', filename = file_name)

#send the email via gmail smtp
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    smtp.login('kehindeolaide24@gmail.com', 'viaaoroaewvexlpc')
    smtp.send_message(msg)

print('email sent successfully')

email sent successfully
